In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from pathlib import Path
import gc
from zipfile import ZipFile
from unidecode import unidecode

project_dir = Path("notebook.ipynb").resolve().parents[0]

In [8]:
initial_data = pd.read_csv(f'{project_dir}/backup/taxa_analfabetismo_maiores_quinze_anos_2010.csv', sep='\n',encoding='latin-1',header=None)
initial_data = initial_data[0].str.split(';', expand=True)

var_name = initial_data[0][1][7:]
var_name = unidecode(var_name).lower().replace(' ','_').replace('/','_').replace(':','')
initial_data.columns = initial_data[initial_data[0]== 'Município'].values.tolist()
initial_data =initial_data.drop([0,1,2,3,4,5], axis=0).reset_index(drop=True)
initial_data = initial_data.drop(columns=['latitude','longitude'])
for column in initial_data.columns[2:]:
    column_name = re.sub(r'\D*','',column)
    initial_data = initial_data.rename(columns={column:f'{column_name}'})


#var_name = re.findall(r'^[A-Za-z_]*','pib_per_capita_2010_2018.csv')[0][:-1]

/home/thiago/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


TypeError: expected string or bytes-like object

In [44]:
test = pd.read_csv(f'{project_dir}/backup/taxa_analfabetismo_maiores_quinze_anos_2010.csv', sep='\n',encoding='latin-1',header=None)
test = test[0].str.split(';', expand=True)
test[test[0]== 'Município'].values.tolist()

[['Município', 'ibge', 'latitude', 'longitude', '2010 (%)']]

In [45]:
test.iloc[5].tolist()

['Município', 'ibge', 'latitude', 'longitude', '2010 (%)']

In [46]:
column_name =test[0][1][7:]
column_name = unidecode(column_name).lower().strip().replace(' ','_').replace('/','_').replace(':','')
column_name

'__educacao_taxa_de_analfabetismo_de_pessoas_de_15_anos_ou_mais'

In [73]:
# lẽ todos os arquivos .zip e descompacta na pasta raw_data:
for archive in os.listdir(f'{project_dir}/raw_data'):
    with ZipFile(f'{project_dir}/raw_data/{archive}','r') as zipfile:
        zipfile.extractall(f'{project_dir}/data/')

total=0

"""initial_data = pd.read_csv('data/pib_per_capita_2010_2018.csv', sep=';',decimal=',', encoding='latin-1')

var_name = initial_data.columns[0][10:]
var_name = unidecode(var_name).lower().replace(' ','_')
initial_data.columns = initial_data.iloc[4]
initial_data =initial_data.drop([0,1,2,3,4], axis=0).reset_index(drop=True)
initial_data = initial_data.drop(columns=['latitude','longitude'])
for column in initial_data.columns[2:]:
    column_name = re.sub(r'\D*','',column)
    initial_data = initial_data.rename(columns={column:f'{column_name}'})

melted_data = initial_data.melt(id_vars=['Município','ibge'], var_name=f'ano', value_name=f'{var_name}_value')
melted_data = melted_data.astype({'ibge':'int','ano':'int'})
"""


#print(f'Tamanho inicial: {melted_data.shape}')
for file in os.listdir(f'{project_dir}/data/'):
    if file == 'TarifaMediaFornecimento.csv':
        continue
    else:
        data = pd.read_csv(f'{project_dir}/data/{file}',sep='\n', encoding='latin-1', header=None)
        data = data[0].str.split(';', expand=True)
        data = data.reset_index(drop=True)
        var_name = data[0][1][7:]
        var_name = unidecode(var_name).lower().strip().replace(' ','_').replace('/','_').replace(':','')
        data.columns = data.iloc[5].tolist()
        data = data.drop([0,1,2,3,4,5], axis=0).reset_index(drop=True)
        data = data.drop(columns=['latitude','longitude'])
        for column in data.columns[2:]:
            column_name = re.sub(r'\D*','',column)
            data = data.rename(columns={column:f'{column_name}'})
        
        data_melted = data.melt(id_vars=['Município','ibge'], var_name=f'ano', value_name=f'{var_name}_value')
        data_melted.to_csv(f'{project_dir}/melted_data/{var_name}_value.csv', sep=';', decimal=',', encoding='latin-1', index=False)
    total+= 1

melted_data = pd.read_csv(f'{project_dir}/melted_data/__educacao_ensino_fundamental_taxa_de_abandono_total_value.csv', header=None, sep='\n',encoding='latin-1')
melted_data = melted_data[0].str.split(';', expand=True)
melted_data.columns= melted_data.iloc[0]
melted_data = melted_data.drop([0], axis=0)
melted_data = melted_data.astype({'ibge':'str','ano':'str'})


for file in os.listdir(f'{project_dir}/melted_data/'):
    if file =='__educacao_ensino_fundamental_taxa_de_abandono_total_value.csv':
        continue
    df= pd.read_csv(f'{project_dir}/melted_data/{file}', header=None, sep='\n', encoding='latin-1')
    print(file)
    df = df[0].str.split(';', expand=True)
    df.columns = df.iloc[0]
    df = df.drop([0], axis=0)
    df = df.astype({'ibge':'str','ano':'str'})
    melted_data = melted_data.merge(df, on=['ibge','ano'], suffixes=('','_D'), how='outer')
    melted_data = melted_data.drop(melted_data.filter(regex='_D$').columns.tolist(), axis=1)
    melted_data.to_csv(f'{project_dir}/preliminar_dataset.csv', sep=';',decimal=',', encoding='latin-1')

final_dataset = melted_data
final_dataset.to_csv(f'{project_dir}/final_dataset.csv', sep=';',decimal=',', encoding='latin-1')
print(f'Tamanho final: {final_dataset.shape}')

__educacao_ensino_medio_numero_de_concluintes_total_value.csv
__educacao_ensino_fundamental_numero_de_concluintes_total_value.csv
__educacao_ensino_fundamental_taxa_de_reprovacao_total_value.csv
__educacao_ensino_medio_taxa_de_abandono_total_value.csv
__educacao_educacao_superior_concluintes__total_value.csv
__educacao_ensino_fundamental_taxa_de_aprovacao_total_value.csv
Tamanho final: (14910, 10)


In [74]:
final_dataset

,Município,ibge,ano,__educacao_ensino_fundamental_taxa_de_abandono_total_value,__educacao_ensino_medio_numero_de_concluintes_total_value,__educacao_ensino_fundamental_numero_de_concluintes_total_value,__educacao_ensino_fundamental_taxa_de_reprovacao_total_value,__educacao_ensino_medio_taxa_de_abandono_total_value,__educacao_educacao_superior_concluintes__total_value,__educacao_ensino_fundamental_taxa_de_aprovacao_total_value
0,Aceguá,4300034,1999,-,-,-,-,-,NaN,-
1,Água Santa,4300059,1999,"2,8",31,69,"8,2","5,1",NaN,"89,0"
2,Agudo,4300109,1999,"3,3",83,186,"19,1","17,2",NaN,"77,6"
3,Ajuricaba,4300208,1999,"2,4",90,135,"12,0","8,2",NaN,"85,6"
4,Alecrim,4300307,1999,"2,8",92,175,"10,5","2,3",NaN,"86,7"
...,...,...,...,...,...,...,...,...,...,...
14905,NaN,4323606,2020,NaN,12,NaN,NaN,NaN,NaN,NaN
14906,NaN,4323705,2020,NaN,24,NaN,NaN,NaN,NaN,NaN
14907,NaN,4323754,2020,NaN,36,NaN,NaN,NaN,NaN,NaN
14908,NaN,4323770,2020,NaN,21,NaN,NaN,NaN,NaN,NaN


In [72]:
#for melt in os.listdir(f'{project_dir}/melted_data/'):
#    print(melt)

a = pd.read_csv(f'{project_dir}/melted_data/__educacao_ensino_fundamental_taxa_de_abandono_total_value.csv', header=None, sep='\n',encoding='latin-1')
a = a[0].str.split(';', expand=True)
a.columns = a.iloc[0]
a = a.drop([0], axis=0)
a.replace('-', np.nan)

,Município,ibge,ano,__educacao_ensino_fundamental_taxa_de_abandono_total_value
1,Aceguá,4300034,1999,NaN
2,Água Santa,4300059,1999,"2,8"
3,Agudo,4300109,1999,"3,3"
4,Ajuricaba,4300208,1999,"2,4"
5,Alecrim,4300307,1999,"2,8"
...,...,...,...,...
10433,Vista Alegre do Prata,4323606,2019,"0,0"
10434,Vista Gaúcha,4323705,2019,"0,0"
10435,Vitória das Missões,4323754,2019,"0,0"
10436,Westfalia,4323770,2019,"3,9"
